**When sales team in bank hands over data to business intelligence department, the BI department digs deep into the data to get the customers list for whom they should pitch the product first
If a proper EDA is not followed, it is not possible to access the list of customers for whom sales team should contact first.
As from business part certain parameters are defined in order to set constraints to get the high priority list of customers for sales pitching
If not done in a systematic manner, with taking into account business parameters, the cost may increase as sales team may start contacting customers with less probability of buying the product.
The main focus of this notebook is to access the list of  those customers who have high probability of buying the product**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data1=pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')

In [ ]:
data1.head()

In [ ]:
data1.info()

In [ ]:
data1.isnull().sum()

In [ ]:
data1.isnull().mean()

In [ ]:
data1.describe()

In [ ]:
data1.describe(include='O')

In [ ]:
data1.shape

EDA:
    1) VISUALIZE

UNIVIRATE ANALYSIS

In [ ]:
data1['Response'].value_counts()

***CLEAR CASE OF IMBALANCE PROBLEM***

**GENDER**

In [ ]:
sns.countplot('Gender',data=data1,hue='Response')
plt.plot()

In [ ]:
data1.groupby(['Gender'])['Response'].value_counts(normalize=True)*100

**AGE**

In [ ]:
sns.distplot(data1['Age'])
plt.plot()

In [ ]:
data1['Age'].describe()

In [ ]:
def brac(x):
    if (x>=20) & (x<31):
        return '20-30'
    if(x>=31) & (x<41):
        return '31-40'
    if(x>=41) & (x<51):
        return '41-50'
    if(x>=51) & (x<61):
        return '51-60'
    if(x>=61) & (x<71):
        return '61-70'
    if(x>=71) & (x<81):
        return '71-80'
    if(x>=81) & (x<91):
        return '81-90'
        

In [ ]:
data1['AgeBracket']=data1['Age'].apply(brac)

In [ ]:
data1[['Age','AgeBracket']]

In [ ]:
sns.countplot('AgeBracket',data=data1,hue='Response')
plt.plot()

In [ ]:
t1=pd.DataFrame(data1.groupby(['AgeBracket'])['Response'].value_counts(normalize=True)*100)

In [ ]:
t1

***SO age bracket of 31-40 and 41-50 and even 51-60 are the ones of who should be contacted first
for vehicle insurance premium***

**DRIVING LICENSE**

In [ ]:
pd.DataFrame(data1.groupby(['Driving_License'])['Response'].value_counts(normalize=True))

***people with drivig liscence should be contacted first to pitch the insurance***

**PTRVIOUSLY INSURED**

In [ ]:
pd.DataFrame(data1.groupby(['Previously_Insured'])['Response'].value_counts(normalize=True))

***People who are not previously insured should be contacted first***

**VEHICLE AGE**

In [ ]:
data1['Vehicle_Age'].unique()

In [ ]:
pd.DataFrame(data1.groupby(['Vehicle_Age'])['Response'].value_counts(normalize=True))

***>2 years and 1-2 years should be contacted first***

**VEHICLE DAMAGE**

In [ ]:
data1['Vehicle_Damage'].unique()

In [ ]:
pd.DataFrame(data1.groupby(['Vehicle_Damage'])['Response'].value_counts(normalize=True))

***People who have damaged the vehicle should be contacted first***

**ANNUAL PREMIUM**

In [ ]:
data1['Annual_Premium'].unique()

In [ ]:
t3=pd.DataFrame(data1.groupby(['Annual_Premium'])['Response'].value_counts(normalize=True))

In [ ]:
t3

In [ ]:
data1['Annual_Premium'].describe()

In [ ]:
data3=data1.copy()

In [ ]:
sns.distplot(data1['Annual_Premium'])
sns.relplot('Response','Annual_Premium',data=data1)

***Annual premium is health premium taken by the customer
lets find out in what range of annual premium does customer go for vehicle insurance***

In [ ]:
data1['quantile_5']=pd.qcut(data1['Annual_Premium'], q=5)

In [ ]:
pd.DataFrame(data1.groupby(['quantile_5'])['Response'].value_counts(normalize=True)*100)

***We can contact all customers in case of annual premium***

***POLICY SALES CAHNNEL***

In [ ]:
pp=pd.DataFrame(data1.groupby(['Policy_Sales_Channel'])['Response'].value_counts(normalize=True)*100)

In [ ]:
pp

ENCODING RARE LABELS IN POLICY SALES CHANNEL

In [ ]:
def find_non_rare_labels(df, variable, tolerance):
    
    temp = df.groupby([variable])[variable].count() / len(df)
    
    non_rare = [x for x in temp.loc[temp>tolerance].index.values]
    
    return non_rare

In [ ]:
def rare_encoding(new, variable, tolerance):
    frequent_cat = find_non_rare_labels(data3, variable, tolerance)

    # re-group rare labels
    data3[variable] = np.where(data3[variable].isin(
        frequent_cat), data3[variable], 'Rare')

    return data3

In [ ]:
for variable in ['Policy_Sales_Channel']:
    
     data3= rare_encoding(data3, variable, 0.01)

In [ ]:
for col in ['Policy_Sales_Channel']:

    temp_df = pd.Series(data3[col].value_counts() / len(data3) )

    # make plot with the above percentages
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # add a line at 5 % to flag the threshold for rare categories
    fig.axhline(y=0.01, color='red')
    fig.set_ylabel('Percentage of houses')
    plt.show()

In [ ]:
pd.DataFrame(data3.groupby(['Policy_Sales_Channel'])['Response'].value_counts(normalize=True)*100)

***For policy sales channel BY TAKING greater than 20% chance we should contact customers which are contacted through
154,156,157 sales channel***

**VINTAGE**

In [ ]:
data1['Vintage'].plot(kind='hist')

In [ ]:
data1['Vin_q']=pd.cut(data1['Vintage'], bins=10)

In [ ]:
pd.DataFrame(data1.groupby(['Vin_q'])['Response'].value_counts(normalize=True)*100)

***In vintage also we can include all***

In [ ]:
sns.distplot(data1['Vintage'])
sns.relplot('Response','Vintage',data=data1)

**REGION CODE**

In [ ]:
for col in ['Region_Code']:

    temp_df = pd.Series(data3[col].value_counts() / len(data3) )

    # make plot with the above percentages
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # add a line at 5 % to flag the threshold for rare categories
    fig.axhline(y=0.05, color='red')
    fig.set_ylabel('Percentage of houses')
    plt.show()

In [ ]:
def find_non_rare_labels(df, variable, tolerance):
    
    temp = df.groupby([variable])[variable].count() / len(df)
    
    non_rare = [x for x in temp.loc[temp>tolerance].index.values]
    
    return non_rare

In [ ]:
find_non_rare_labels(data3, 'Region_Code', 0.02)

In [ ]:
def rare_encoding(new, variable, tolerance):
    frequent_cat = find_non_rare_labels(data3, variable, tolerance)

    # re-group rare labels
    data3[variable] = np.where(data3[variable].isin(
        frequent_cat), data3[variable], 'Rare')

    return data3

In [ ]:
for variable in ['Region_Code']:
    
     data3= rare_encoding(data3, variable, 0.01)

In [ ]:
for col in ['Region_Code']:

    temp_df = pd.Series(data3[col].value_counts() / len(data3) )

    # make plot with the above percentages
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # add a line at 5 % to flag the threshold for rare categories
    fig.axhline(y=0.01, color='red')
    fig.set_ylabel('Percentage of houses')
    plt.show()

In [ ]:
pd.DataFrame(data3.groupby(['Region_Code'])['Response'].value_counts(normalize=True)*100)

***for region code we can use greater than 10% chance
region 11,18,20,29,3,25,39,41***

**Customer list that has to be contatced first**

In [ ]:
data_contact=data3[(data3['Driving_License']==1) & 
      (data3['Previously_Insured']==0) &
     (data3['Vehicle_Age']=='> 2 Years') & (data3['Vehicle_Damage']=='Yes') & 
      ((data3['AgeBracket']=='31-40') | (data3['AgeBracket']=='41-50') | (data3['AgeBracket']=='51-60')) & 
                  ((data3['Region_Code']=='11.0') | (data3['Region_Code']=='18.0') |
                  (data3['Region_Code']=='20.0') | (data3['Region_Code']=='29.0') |
                  (data3['Region_Code']=='3.0') | (data3['Region_Code']=='25.0') |
                  (data3['Region_Code']=='39.0') | (data3['Region_Code']=='41.0')) & 
                  ((data3['Policy_Sales_Channel']=='154.0') | (data3['Policy_Sales_Channel']=='156.0') |
                  (data3['Policy_Sales_Channel']=='157.0'))]

In [ ]:
data_contact.head()

In [ ]:
data_contact.shape

***SO from 381109 customers we have boiled down the list to 106,
the sales department can use this list and contact them first***

In [ ]:
cor1=data1.corr()

In [ ]:
plt.figure(figsize = (16,10))
sns.heatmap(cor1,linewidths=1,annot=True)
plt.plot()

***Here we have to take care of Recall more than Precision
By taking care of Recall, We say that we want to increase Our True positives and reduce False neagtive
We dont want to miss out customers who have actually responed 1***
*
**We also want to take care of type 1 error
that is we dont want to classify who has response as 0 to response of 1
it may increase our cost on business side, as calling may increase***

***but recall has higher importance here***

In [ ]:
data2=data1.copy()

In [ ]:
data2.drop(['AgeBracket','quantile_5','Vin_q'],axis=1,inplace=True)

In [ ]:
data2.shape

In [ ]:
data_test=pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/test.csv')

In [ ]:
data_test.shape

In [ ]:
#data_test.drop(['Annual_Premium','Vintage'],axis=1,inplace=True)

In [ ]:
new=pd.concat([data2,data_test],axis=0)

In [ ]:
new['Vintage']=new['Vintage']/365

In [ ]:
for col in ['Region_Code']:

    temp_df = pd.Series(new[col].value_counts() / len(new) )

    # make plot with the above percentages
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # add a line at 5 % to flag the threshold for rare categories
    fig.axhline(y=0.05, color='red')
    fig.set_ylabel('Percentage of houses')
    plt.show()

In [ ]:
def find_non_rare_labels(df, variable, tolerance):
    
    temp = df.groupby([variable])[variable].count() / len(df)
    
    non_rare = [x for x in temp.loc[temp>tolerance].index.values]
    
    return non_rare

In [ ]:
find_non_rare_labels(new, 'Region_Code', 0.02)

In [ ]:
[x for x in new['Region_Code'].unique(
) if x not in find_non_rare_labels(new, 'Region_Code', 0.02)]

In [ ]:
new1=new.copy()

In [ ]:
def rare_encoding(new, variable, tolerance):
    frequent_cat = find_non_rare_labels(new1, variable, tolerance)

    # re-group rare labels
    new1[variable] = np.where(new1[variable].isin(
        frequent_cat), new1[variable], 'Rare')

    return new1

In [ ]:
for variable in ['Region_Code']:
    
     new1= rare_encoding(new1, variable, 0.01)

In [ ]:
for col in ['Region_Code']:

    temp_df = pd.Series(new1[col].value_counts() / len(new1) )

    # make plot with the above percentages
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # add a line at 5 % to flag the threshold for rare categories
    fig.axhline(y=0.01, color='red')
    fig.set_ylabel('Percentage of houses')
    plt.show()

In [ ]:
g1=pd.DataFrame((new1.groupby(['Region_Code'])['Response'].value_counts(normalize=True)*100).sort_values(ascending=False))

In [ ]:
for col in ['Policy_Sales_Channel']:

    temp_df = pd.Series(new[col].value_counts() / len(new) )

    # make plot with the above percentages
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # add a line at 5 % to flag the threshold for rare categories
    fig.axhline(y=0.01, color='red')
    fig.set_ylabel('Percentage of houses')
    plt.show()

In [ ]:
def find_non_rare_labels(df, variable, tolerance):
    
    temp = df.groupby([variable])[variable].count() / len(df)
    
    non_rare = [x for x in temp.loc[temp>tolerance].index.values]
    
    return non_rare

In [ ]:
find_non_rare_labels(new1, 'Policy_Sales_Channel', 0.01)

In [ ]:
def rare_encoding(new, variable, tolerance):
    frequent_cat = find_non_rare_labels(new1, variable, tolerance)

    # re-group rare labels
    new1[variable] = np.where(new1[variable].isin(
        frequent_cat), new1[variable], 'Rare')

    return new1

In [ ]:
for variable in ['Policy_Sales_Channel']:
    
     new1= rare_encoding(new1, variable, 0.01)

In [ ]:
for col in ['Policy_Sales_Channel']:

    temp_df = pd.Series(new1[col].value_counts() / len(new1) )

    # make plot with the above percentages
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # add a line at 5 % to flag the threshold for rare categories
    fig.axhline(y=0.01, color='red')
    fig.set_ylabel('Percentage of houses')
    plt.show()

In [ ]:
pc=pd.DataFrame((new1.groupby(['Policy_Sales_Channel'])['Response'].value_counts(normalize=True)*100).sort_values(ascending=False))

In [ ]:
pc

In [ ]:
new1.dtypes

In [ ]:
#from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder

In [ ]:
#dl=dict(new['Driving_License'].value_counts())

In [ ]:
#new['Driving_License']=new['Driving_License'].replace(dl)

In [ ]:
#a=dict(new['Region_Code'].value_counts())

In [ ]:
#new['Region_Code']=new['Region_Code'].replace(a)

In [ ]:
new1['Vehicle_Age']=new1['Vehicle_Age'].replace({'< 1 Year':0,'1-2 Year':1,'> 2 Years':2})

In [ ]:
new1['Vehicle_Damage']=new1['Vehicle_Damage'].replace({'Yes':1,'No':0})

In [ ]:
new1['Gender']=new1['Gender'].replace({'Male':1,'Female':0})

In [ ]:
new1.head()

In [ ]:
!pip install feature-engine
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

In [ ]:
#new.columns

In [ ]:
new1.dtypes

In [ ]:
new1['Region_Code'].unique()

In [ ]:
#new1['Region_Code']=new1['Region_Code'].replace({'Rare':0})

In [ ]:

#new1=new1.astype({'Region_Code':float})

**OUTLIER Treatment**

In [ ]:
new1.describe()

In [ ]:
import scipy.stats as stats

In [ ]:
# function to create histogram, Q-Q plot and
# boxplot. We learned this in section 3 of the course


def diagnostic_plots(df, variable):
    # function takes a dataframe (df) and
    # the variable of interest as arguments

    # define figure size
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.distplot(df[variable], bins=30)
    plt.title('Histogram')

    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')

    # boxplot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')

    plt.show()

In [ ]:
diagnostic_plots(new, 'Age')

In [ ]:
diagnostic_plots(new, 'Annual_Premium')

In [ ]:
diagnostic_plots(new, 'Vintage')

In [ ]:
def find_skewed_boundaries(df, variable, distance):

    # Let's calculate the boundaries outside which sit the outliers
    # for skewed distributions

    # distance passed as an argument, gives us the option to
    # estimate 1.5 times or 3 times the IQR to calculate
    # the boundaries.

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary

In [ ]:
ap_upper_limit, ap_lower_limit = find_skewed_boundaries(new, 'Annual_Premium', 1.5)
ap_upper_limit, ap_lower_limit

In [ ]:
# Now let's replace the outliers by the maximum and minimum limit

new['Annual_Premium']= np.where(new['Annual_Premium'] > ap_upper_limit, ap_upper_limit,
                       np.where(new['Annual_Premium'] < ap_lower_limit, ap_lower_limit, new['Annual_Premium']))

In [ ]:
diagnostic_plots(new, 'Annual_Premium')

In [ ]:
new['Annual_Premium'].describe()

In [ ]:
ohe_enc = OneHotCategoricalEncoder(
    top_categories=None, # we can select which variables to encode
    drop_last=True) # to return k-1, false to return k


ohe_enc.fit(new1)

In [ ]:
tmp = ohe_enc.transform(new1)

tmp.head()

In [ ]:
#tmp.drop('Driving_License',axis=1,inplace=True)

In [ ]:
tmp.drop('id',axis=1,inplace=True)

In [ ]:
Train=tmp.iloc[0:381109]

In [ ]:
Test=tmp.iloc[381109:]

In [ ]:
X=Train.drop('Response',axis=1)

In [ ]:
Y=Train['Response']

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
os =  RandomOverSampler(0.70)

In [ ]:
X_train_res, y_train_res = os.fit_sample(X, Y)

In [ ]:
X_train_res.shape,y_train_res.shape

In [ ]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y)))
print('Resampled dataset shape {}'.format(Counter(y_train_res)))

**CHI SQUARE FOR FEATURE SELECTION**

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
ordered_rank_features=SelectKBest(score_func=chi2,k=5)
ordered_feature=ordered_rank_features.fit(X_train_res,y_train_res)

In [ ]:

dfscores=pd.DataFrame(ordered_feature.scores_,columns=["Score"])
dfcolumns=pd.DataFrame(X.columns)



In [ ]:
features_rank=pd.concat([dfcolumns,dfscores],axis=1)

In [ ]:
features_rank.columns=['Features','Score']


In [ ]:
X_train_res.shape

In [ ]:
features_rank.nlargest(15,'Score')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train_res,
                                                    y_train_res,
                                                    test_size=0.3,
                                                    random_state=0)

X_train.shape, X_test.shape

In [ ]:
Test.drop('Response',axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_squared_log_error, roc_auc_score

In [ ]:
test1=pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/test.csv')

In [ ]:
id1=test1['id']

In [ ]:
from lightgbm import LGBMClassifier
lgbcl = LGBMClassifier(boosting_type='gbdt',n_estimators=500,depth=10,learning_rate=0.04,objective='binary',metric='auc',
                 colsample_bytree=0.5,reg_lambda=2,reg_alpha=2,random_state=294,n_jobs=-1)
lgbcl= lgbcl.fit(X_train, Y_train,eval_metric='auc',verbose=2)
y_lgb = lgbcl.predict(X_test)
probs_tr = lgbcl.predict_proba(X_train)[:, 1]
probs_te = lgbcl.predict_proba(X_test)[:, 1]
print(roc_auc_score(Y_train, probs_tr))
print(roc_auc_score(Y_test, probs_te))

In [ ]:
lgbcl1=lgbcl.fit(X_train_res,y_train_res)

In [ ]:
lgbcl1

In [ ]:
lgb_pred= lgbcl1.predict_proba(Test)[:, 1]

In [ ]:
lgb_pred

In [ ]:
lgb=pd.concat([pd.DataFrame(id1),pd.DataFrame(lgb_pred)],axis=1)

In [ ]:
lgb2=lgb.copy()

In [ ]:
lgb2.rename(columns={0:'Response'},inplace=True)

In [ ]:
lgb2.to_csv('lgb_onehot+label.csv',index=False)

In [ ]:
te=lgbcl.predict(X_test)

In [ ]:
tr=lgbcl.predict(X_train)

In [ ]:
from sklearn.metrics import confusion_matrix
trc=confusion_matrix(Y_train,tr)

In [ ]:

tec=confusion_matrix(Y_test,te)

In [ ]:
recall_train=((trc[0][0])/(trc[0][0]+trc[1][0]))*100

In [ ]:
recall_train

In [ ]:
recall_test=((tec[0][0])/(tec[0][0]+tec[1][0]))*100

In [ ]:
recall_test

In [ ]:
precision_train=((trc[0][0])/(trc[0][0]+trc[0][1]))*100

In [ ]:
precision_train

In [ ]:
precision_test=((tec[0][0])/(tec[0][0]+trc[0][1]))*100

In [ ]:
precision_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf1=RandomForestClassifier()

In [ ]:
cross_val_score(rf1,X_train_res,y_train_res,cv=5,scoring='accuracy')

In [ ]:
rf1.fit(X_train,Y_train)

In [ ]:
probs_tr_rf = rf1.predict_proba(X_train)[:, 1]
probs_te_rf = rf1.predict_proba(X_test)[:, 1]
print(roc_auc_score(Y_train, probs_tr_rf))
print(roc_auc_score(Y_test, probs_te_rf))

In [ ]:
te=rf1.predict(X_test)

In [ ]:
tr=rf1.predict(X_train)


In [ ]:
from sklearn.metrics import confusion_matrix
trc=confusion_matrix(Y_train,tr)

In [ ]:

tec=confusion_matrix(Y_test,te)

In [ ]:
recall_train=((trc[0][0])/(trc[0][0]+trc[1][0]))*100

In [ ]:
recall_train

In [ ]:
recall_test=((tec[0][0])/(tec[0][0]+tec[1][0]))*100

In [ ]:
recall_test

In [ ]:
precision_train=((trc[0][0])/(trc[0][0]+trc[0][1]))*100

In [ ]:
precision_train

In [ ]:
precision_test=((tec[0][0])/(tec[0][0]+trc[0][1]))*100

In [ ]:
precision_test

In [ ]:
F1_train = 2 * (precision_train * recall_train) / (precision_train + recall_train)

In [ ]:
F1_train

In [ ]:
F1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)

In [ ]:
F1_test

In [ ]:
rf_t=rf1.predict_proba(Test)

In [ ]:
rf_t

In [ ]:
rf123=pd.concat([pd.DataFrame(id1),pd.DataFrame(rf_t)],axis=1)

In [ ]:
rf_done=rf123.drop(0,axis=1)

In [ ]:
rf_done.rename(columns={1:'Response'},inplace=True)

In [ ]:
rf_done.to_csv('AR_check.csv',index=False)

In [ ]:
Kindly Like the notebook, as it motivates me :), Thank You!